In [1]:
%run stdPackages.ipynb # this imports a lot of useful packages 
import base
from pyDbs import read, readSets, broadcast # some helpful auxiliary database functions we will rely on 

# The ```lpBlock``` class - Part II: Compilation and execution

We start by importing the database and the instance of the block with the structured arguments:

In [2]:
with open(os.path.join(d['data'],'blockPartI'), "rb") as file:
    block = pickle.load(file)
with open(os.path.join(d['data'],'blockPartI_db'), "rb") as file:
    db = pickle.load(file)

This part II takes you through the compiling process as it may provide some insights into the troubleshooting process - if a model is misspecified. The main compilation function is implemented via the ```__call__(self, execute = None)``` method which can be called using the syntax ```self(execute = None)```. This method runs through four core steps that we go through in the following sections (the kwarg ```execute``` can be passed to only run a subset of the methods):

>[1. compileParameters]: Takes the arguments that we specified in [**part I**](./lpCompiler_PartI.ipynb) and constructs new components.  
>[2. settingsFromCompiled]: Extracts some useful meta-data from the compiled data, e.g. defining constraints/variables and their respective domains in the model.  
>[3. inferGlobalDomains]: Uses the compiled elements to establish global domains ...  
>[4. getDenseArgs]: Performs broadcasting and ... 

## 1. Augmented form linear programming <a id='AugLP'></a>